# Scrape Pubmed To Fastrack Literature Searches
## Lets say you are a med student tasked with doing a lit review, or a scientist interested in certain topics. You could use a plethora of the free tools available (Pubmed, google scholar, elicit, etc), but if you are lazy and want to automate some of the hard work, you could also build a web scraper in Python that will expedite this process for you.
## Here we will go through the construction of a very basic web scraper to dig through Pubmed and return articles and some metadata about them automatically. Our topic will be metformin and its relationship to lactic acidosis

### Install dependecies. Biopython is a set of python tools for biologic computation and you import it using Bio. Entrez is a search engine that provides access to manu biomedical databases

In [88]:
%pip install biopython

from Bio import Entrez
import pandas as pd
import numpy as np
import sys



Note: you may need to restart the kernel to use updated packages.


## For this project we will be using functions. Functions are a tool in Python to neatly package code that you will reuse over and over again. We will be making a 'search' function to probe the database and a 'get_details' functino to retrieve metadata about the identified articles. There is information about the code in commented text. You will see how we call all the functions at the end of the notebook, so if you get confused early just keep going and it will come together

### Search function- takes as input a query, a minimum date, and a maximum date
### The search_setup variable is what we will send to Entrez to get the pubmed IDs. We build the handle by specifying Entrez.esearch(parameter1,parameter2...etc)


In [89]:
# query- keywords you want to search
# min_date- earliest date to start the search
# max_date- latest date to end the search
# number- number of articles to return
def search(query, min_date, max_date, number):
    search_setup = Entrez.esearch(db='pubmed', #db: the database we want to search. Find more about the various database options in table 1 of 'A General Introduction to the E-utilities' paper
            email='gnahas2@uic.edu', # An email is required in case something goes wrong
            sort='relevance', # This decides how you want to sort the identified articles
            retmax=number, # This is the maximum number of articles to return passes as an argument to the function
            mindate = min_date, # This is the initial date of articles you want to search for (passed as argument to the function)
            maxdate = max_date, # This is the latest date of articles you want to search for (passes as argument to the function)
            term=query) # This is the search you will send to the database that got passed to the function (more on this later)
    results = Entrez.read(search_setup) #this stores the results of the database query (pubmed IDs) as a list that we return  
    return results

### This function fetches the all of the metadata associated with the list of pubmed IDs identified from the earlier search function. We will use this data later to build our spreadsheet!


In [80]:
# id_list- list of pubmed IDs that were identified earlier
def fetch_details(id_list):
    ids = ','.join(id_list) # make a new single string of all the ids separated by a comma in id_list to submit
    fetch_setup = Entrez.efetch(db='pubmed',  #db: the database we want to search. Find more about the various database options in table 1 of 'A General Introduction to the E-utilities' paper
            email='gnahas2@uic.edu',  # An email is required in case something goes wrong
            id=ids) # ids we will be searching for in the database
    results = Entrez.read(fetch_setup) #this stores the results of the metadata of the pubmed IDs in a pretty messy format, but one that is accessible to the computer! Read below to see how to extract it
    return results



### This following function can be a bit confusing because there is a lot going on. The goal of this function is to extract the metadata (which is stored in a dictionary) and then save it to a dataframe (another form of data storage in Python- you will learn about these in Module 3-4) so we can write a csv file neatly. 
### The structure of the results from Entrez stores all of the relevant data we care about under the MedlineCitation key. We had to look this up, which is part of coding! See the comments in the code for much more detail about what is going on.

In [85]:
# results- this is the data from the fetch_details function above!
def extract_info(results):
    parsed_articles = [] # we define an empty list that we can store the parsed data we extract in 
    for article in results['PubmedArticle']: # We use a for loop to iterate through all of the articles in our results. 
                                             # Each article and its metadata is hierarchicaly stored with the top of the tree being the 'PubmedArticle'.
                                             # Again, we had to look all this up!
        medline = article['MedlineCitation'] # We now want to extract the rest of the data we care about which is at the next level of the tree. All of this 
                                             # data is contained under the ['MedlineCitation'] key, so as we iterate through every article we want to always keep 
                                             # track and store the Medline info
        pmid = medline['PMID'] # extract the pubmed ID using the key PMID
        title = medline['Article']['ArticleTitle'] # Extract the article title (the title is nested under the article key so you have to do two layers of keys)

        # now we want to store the authors, but we have to be careful. Sometimes the author data ends up in the wrong place or not retrieved during the database query.
        # as such, we implement some checks to make sure we dont run into any errors
        authors = [] # define a new list to store the author names in 
        if 'AuthorList' in medline['Article']: # check if there is even an author list in the current article
            for author in medline['Article']['AuthorList']: # if there is an author list, iterate through it so we can store the names
                last_name = author.get('LastName', '') # get the last name
                fore_name = author.get('ForeName', '') # get the first name 
                if last_name or fore_name: # if we found a last name or a first name, add it to the list we made earlier. This takes care of cases where only a first or last name is found
                    authors.append(last_name + ', ' + fore_name) # append it to the list
        else: # if there was no author list append authors not found
            authors.append('Author(s) not found')

        pub_date = medline['Article']['Journal']['JournalIssue']['PubDate'] # Get the publication date and store it in a variable
        
        # Check to see if there is an abstract
        if 'Abstract' in medline['Article']:
            abstract = ' '.join(medline['Article']['Abstract']['AbstractText']) # the abstract is a list of strings. We want to join them into one string separating them by spaces
        else:
            abstract = 'No abstract available' # Write this if no abstract is available
        
        # Handle cases where no keywords are available
        if 'KeywordList' in medline and len(medline['KeywordList']) > 0: # check to see if there is a keywords value and the length of them is greater than 0
            keywords = [keyword for keyword in medline['KeywordList'][0]] # if so, the store the keywords in a list (this is just a for loop, but its called loop comprehension)
        else:
            keywords = 'No keywords available' # append this if no keywords are found
        
        # Remember when we defined this list earlier? Now we want to append all of our extracted data to this list
        # We store the information in a dictionary as that is very easy to convert into a dataframe for conversion to csv 
        parsed_articles.append({
            'pmid': pmid,
            'title': title,
            'authors': ', '.join(authors),
            'pub_date': pub_date,
            'abstract': abstract,
            'keywords': ', '.join(keywords)
        })
    return pd.DataFrame(parsed_articles) # convert the parsed article to a dataframe and return! Notice the change in tab levels here (we are now outside the for loop)

### Now we are going to set up the experiment and call all the functions we just made! This is where we will get our results and reap the benefits of our hard work

In [92]:
keywords = 'artificial intelligence in medical school education' # These are the keywords that we send to Entrez to query
number = 20 # this is the number of articles we return
date_begin = '2010/01/01' # the date we start searching from
date_end = '2023/05/01' # the date we end the search 

studies = search(keywords, date_begin, date_end, number) # We call the search function and send in our parameters in order to receive the pubmed IDs from our search
studiesIdList = studies['IdList'] # We want to extract the IdList (it is in a dictionary, so we use the key associated with it)
metadata = fetch_details(studiesIdList) # we send the Ids to our fetch_details function to get more information about the identified articles

data = extract_info(metadata) # we extract the information that we received from the fetch_details

data.to_csv(keywords + '.csv', index=False) # we plot our data as a CSV



{'Count': '2892', 'RetMax': '20', 'RetStart': '0', 'IdList': ['33962110', '31829208', '32861647', '32024055', '34476701', '31322909', '33421167', '29980928', '34791021', '34969102', '31980086', '35568690', '33599773', '33739235', '30646206', '36905924', '35640917', '33677250', '35653504', '34763060'], 'TranslationSet': [{'From': 'artificial intelligence', 'To': '"artificial intelligence"[MeSH Terms] OR ("artificial"[All Fields] AND "intelligence"[All Fields]) OR "artificial intelligence"[All Fields]'}, {'From': 'medical school', 'To': '"schools, medical"[MeSH Terms] OR ("schools"[All Fields] AND "medical"[All Fields]) OR "medical schools"[All Fields] OR ("medical"[All Fields] AND "school"[All Fields]) OR "medical school"[All Fields]'}, {'From': 'education', 'To': '"educability"[All Fields] OR "educable"[All Fields] OR "educates"[All Fields] OR "education"[Subheading] OR "education"[All Fields] OR "educational status"[MeSH Terms] OR ("educational"[All Fields] AND "status"[All Fields]) O

## CONGRATULATIONS! You now know how a web scraper works, and hopefully learned about new ways to automate some of the 'boring' stuff.
## As a challenge, try to place the following lines of code in their correct place above. If you do this, you will create an AI powered scraper that summarizes the abstract for you automatically! Good luck!

In [ ]:
# %pip install bert-extractive-summarizer
# from summarizer import Summarizer

# def summarize_abstract(abstract):
#     model = Summarizer()
#     return model(abstract, num_sentences=3)

# summary = summarize_abstract(abstract)

# parsed_articles.append({
#     'pmid': pmid,
#     'title': title,
#     'authors': ', '.join(authors),
#     'pub_date': pub_date,
#     'abstract': abstract,
#     'summary' : summary,
#     'keywords': ', '.join(keywords)
# })
    